# 네이버 뉴스, 블로그 Url - 크롤링 수집 작업 과정
네이버 뉴스와 블로그 Url 수집 작업은 검색량의 제한이 설정되어 있어 매일 각 키워드에 해당하는 Url을 수집한 뒤, 분석 시작시 수집된 일자별 Url의 본문 내용을 수집하는 순서로 진행하였습니다.

1. 네이버 뉴스 검색 키워드
- 카카오톡 개편, 카카오톡 업데이트, 카톡, 네이트온, 카카오

2. 네이버 블로그 검색 키워드
- 카카오톡 업데이트, 카카오톡 개편, 카톡 업데이트, 카톡 개편, 네이트온, 라인
- 11월 9일 추가 : 카카오톡, 카톡

*[!] 네이버 뉴스와 블로그는 매주 목요일(+α)을 기준으로 크롤링에 사용되는 조건값들을 갱신되기 때문에 코드 조정이 필요하였습니다.*

*[!] 코렙에서 크롤링 진행 중 Chrome 버전이 맞지 않아 크롤링이 진행되지 않는 호환성 문제가 발생되어 매일 크롤링을 진행할 때 Chrome을 새로 다운로드 받아 설치하는 것으로 문제를 해결하였습니다.*

In [ ]:
# 셀레니움 사용을 위한 설정 작업(공통)

# 1. 셀레니움 라이브러리를 설치합니다.
!pip install selenium

# 2. 코랩의 리눅스 환경에 크롬 브라우저와 웹 드라이버 설치
!apt-get update

# 3. 최신 Google Chrome 브라우저를 다운로드하고 설치 - 호환성 문제 해결용
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install --yes ./google-chrome-stable_current_amd64.deb

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 4. 셀레니움이 코랩의 웹 드라이버를 사용할 수 있도록 설정(별도 창 오픈 안되고 처리되도록)
options = webdriver.ChromeOptions()
options.add_argument('--headless') # << 요기
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')

print("✅ WebDriver 세션이 성공적으로 생성되었습니다. 크롤링을 시작할 수 있습니다.")

## [Phase 1-1] 네이버 뉴스 URL 수집

1. 프로젝트 개요

해당 파일은 '카카오톡 업데이트 사용자 피드백 분석' 프로젝트의 데이터 수집 단계 중, 네이버 뉴스의 게시물 URL을 수집하는 과정을 담고 있습니다.

네이버 뉴스 검색 결과 페이지의 '기사 덩어리(Cluster)'라는 독특하고 복잡한 구조에 대응하기 위해 Selenium 기반의 맞춤형 스크래핑 로직을 구축했습니다.

2. 주요 로직 및 문제 해결 과정

네이버 뉴스 검색 결과는 하나의 '대표 기사'와 다수의 '관련 뉴스'가 묶인 '기사 덩어리' 형태로 제공됩니다. 모든 관련 기사의 URL을 누락 없이 수집하기 위해 다음과 같은 전략을 적용했습니다.

- 동적 페이지 전체 로딩: Selenium을 활용하여 목표한 수의 '기사 덩어리'가 모두 로드될 때까지 페이지 스크롤을 자동 반복했습니다.
- '대표/관련' 구조 파싱: 로드된 페이지 전체를 대상으로, 각 '기사 덩어리'를 먼저 찾고, 그 안에서 다시 '대표 기사' 영역과 '관련 뉴스' 영역을 구분하여 각각의 제목, 언론사, URL 정보를 정교하게 추출하는 '2단계 파싱(Two-step Parsing)' 로직을 구현했습니다.
- 안정적인 중간 저장: 수백 개의 '기사 덩어리'를 처리하는 과정에서 발생할 수 있는 데이터 유실을 방지하기 위해, 일정 개수(save_interval)의 덩어리를 처리할 때마다 수집된 URL 목록을 CSV 파일에 주기적으로 중간 저장하는 기능을 포함했습니다.
- 실행 환경 안정화: 코랩(Colab) 환경의 크롬 드라이버 호환성 문제에 대응하기 위해, 스크립트 실행 시 항상 최신 버전의 크롬을 설치하는 로직을 추가하여, 장시간의 수집 작업이 안정적으로 수행될 수 있는 환경을 구축했습니다.

3. 결과물

스크립트 내 입력한 키워드별로 '작성자, 날짜, 제목, URL' 정보가 포함된 CSV 파일이 생성됩니다.

In [ ]:
import time
import pandas as pd
import re

# 드라이버 경로를 지정 및 실행
browser = webdriver.Chrome(options=options)
wait = WebDriverWait(browser, 10)

# 검색 키워드 입력
keyword = "카카오"
max_articles_to_collect = 200
backup_filename = "251120_naver_news_links_backup1.csv"
collected_news = []
processed_urls = set()

print(f"'{keyword}' 키워드로 최대 {max_articles_to_collect}개의 기사 덩어리 수집을 시작합니다. (관련도순)")

url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={keyword}"
browser.get(url)
time.sleep(2)

# 스크롤하여 모든 기사 로드 - 목표 수량의 기사 '덩어리'가 로드될 때까지 스크롤합니다.
while True:
    try:
        # 기사 블록 코드 설정
        # UD3s7I8expz3WkwQ004B > IgJMhH4Xrhuw6jctOE_e > KetQP_LKIbcM61v6xsK_ > vs1RfKE1eTzMZ5RqnhIv > o2YzmBxKhEuKOM4uNxFM > OXZTIahLwg7zu2s4BjcD > dnrvM2QEkuzd3V9zdADN
        # zc470m0U4oRqtEfZ3YqX > C27ncwLnr9ZhoFd7p9eW > dHVkS02MdHAPCLuLeYE_ > i3viLIysNP0cK6eVSE_t > Lfnc2BS3iprGnUjPmaau > nFxvxHv3aFvb8orT0c7L > tyhUp9y3pNUE2zu1vg4o
        article_clusters = browser.find_elements(By.XPATH, "//div[contains(@class, 'shjpbJ1U8dIwWXdtD0kq')]")

        print(f"스크롤 다운... 현재 로드된 기사 덩어리 수: {len(article_clusters)}개")

        if len(article_clusters) >= max_articles_to_collect:
            print(f"목표 수량({max_articles_to_collect}개) 이상의 기사 덩어리가 로드되어 스크롤을 중단합니다.")
            break

        last_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            print("페이지의 끝에 도달하여 스크롤을 중단합니다.")
            break

    except Exception as e:
        print(f"스크롤 중 오류 발생: {e}")
        break

# 스크롤 완료 후, 로드된 모든 기사 정보 일괄 수집
print("\n" + "="*50)
print("스크롤 완료. 로드된 모든 '기사 덩어리'를 분석하여 중간 저장합니다.")
print("="*50)

# 기사 블록 코드를 그대로 입력
final_article_clusters = browser.find_elements(By.XPATH, "//div[contains(@class, 'shjpbJ1U8dIwWXdtD0kq')]")
print(f"총 {len(final_article_clusters)}개의 '기사 덩어리'를 찾았습니다. 정보 추출을 시작합니다.")

# 중간 저장 설정
save_interval = 100
is_first_save = True
temp_list = [] # 중간 저장을 위한 임시 리스트

for i, cluster in enumerate(final_article_clusters):
    try:
        # 1. 대표 기사 정보 수집(대표 기사의 링크 찾기)
        # Kv_vr2pgKJ1YQu5GxPEk > g142CmJWlznnvvbvmk68 > oE0MWYkMadhMOexVagqP > VVZqvAlvnADQu8BVMc2n > r5Erm7sXBDLtcwB4qpy7 > nwfSPmNIlDyRhvW6TUmz > MhHMvACwlVBbjWsd7yi3
        # SR2LrlI9g02spd3asYd0 > xaRANlI1WEmTnsgGH3eP > EwZelK7klfw43H0sizCm > FjllvMNpyI4wLwvZ4zbu > sS0WyM73AIK9pz1Yqyqq > moM44hE6Je7O8nL1iBI9 > eRlwIV3EhjKD0fYL3LaQ
        main_link_element = cluster.find_element(By.XPATH, ".//a[contains(@class, 'yuu64AGiOBzaFbBUUZbL')]")
        href = main_link_element.get_attribute('href')

        if href not in processed_urls:
            press_name = cluster.find_element(By.XPATH, ".//span[contains(@class, 'sds-comps-profile-info-title-text')]").text

            # 날짜 정보 수집(대표 기사와 SUB 기사 모두 코드 동일함)
            # Ltv7sJRJrOOmpZjSEje_ > wcxswLWlceUCHTW8Wb6J > U1zN1wdZWj0pyvj9oyR0 > FeA7N0BhBgO_D8kbFxJF > PgHY7RzLrrRbaQRE6rd6 > fgTXvYZMrLfEWoFdUCTw > MA8c8CAJTXR5VZfzQIye
            # pwnjCGK9f7mL1YDI1uYb > SaoU38edkXdPztmLceJP > PMb1a1DEernbhZd2MxiT > Ma1nB_t9KiMSHtLJMRq5 > _WbCGyUrBcsoRc8OCWZC > FGGxCHMwgwpUxnJbY5q7
            date_elements = cluster.find_elements(By.XPATH, ".//div[contains(@class, 'r8Zk0bSoBfwe23GKUAiO')]")
            date_str = date_elements[0].text if date_elements else "날짜 정보 없음"

            temp_list.append({
                "언론사": press_name, "날짜_원문": date_str, "제목": main_link_element.text,
                "URL": href, "구분": "대표 기사"
            })
            processed_urls.add(href)

        # 2. Sub 기사 정보 수집
        try:
            # sub 기사를 감싸는 영역 찾기
            # kKg41qrHvplVksYUiHBW > Me1nXk2Sj0arGqU1zX5w > bVTBojHRFWtr0ZbV6C5y > FswuEMo_eVFKY6_b3tld > OF07cvLiENTY1VKtFc5A > EwAFgcAS665NvfKxDROd
            # Ei1cZ0jRRr6RYsXhyTPO > yvf5xJP1YqnVwu_4khLV > JQDuwtyvXrFV747dop5H > BzIGBxg89LPpHBcTjqMA > PrUOhta1LIOWOfmEvQmw
            sub_links_area = cluster.find_element(By.XPATH, ".//div[contains(@class, 'YZ4O2FJngrWp9COsZhRU')]")

            # sub 기사 한개씩의 영역을 찾기
            # qpBnmqdPyZ4VYb3aEgnw > wHeGm6Q2cZzREeeoGAek > pXtzVXJhhzmMSeJWgamo > sNqs4qlOsawpNHKodxwe > o_f72EnDpGiEMgGEWQbn > RgzdKee2MqFRfya9VhXk
            # sTfU4DwVahUIvJQ9rT8k > LSwpmnQ0XBvFUIVyu5aw > NduKo_r15nFFL3c3ldcs > Ri9R136Wu9IqTDlzHSpZ > JckXcPhmnCSGElhUB9mQ
            sub_articles = sub_links_area.find_elements(By.XPATH, ".//div[contains(@class, 'x8YkymRY1k1U89_l2ofe')]")

            for sub_article in sub_articles:
                # sub 기사의 링크 찾기
                # sOCNRZynP_svsS9PHNyP > pk_3C5bLJhKIBI3qxn7Q > LQ84J3qaJu0K7yYnc7ei > ro0zKDMMT_0keyhkUqcD > oOo9vZQJs74avFqK6iAQ > rW6cNpojrBPcetkA2xLA
                # gE9uY284DgZyTd9TGAfk > PHurNvfBgVciz1x0Xh7G > VqC9n9fwv6dcA8ebYm4l > IVReVs7J6QJhAhvZgk5Q > kBfZiHy29ZtrLZnmujW6
                sub_link_element = sub_article.find_element(By.XPATH, ".//a[contains(@class, 'jXFuXTqqS_JwIbn0Gl4Q')]")
                sub_href = sub_link_element.get_attribute('href')

                if sub_href not in processed_urls:
                    sub_title = sub_link_element.text
                    sub_press = sub_article.find_element(By.XPATH, ".//span[contains(@class, 'sds-comps-profile-info-title-text')]").text
                    # 대표 기사에서 찾은 날짜 코드 입력
                    sub_date_elements = sub_article.find_elements(By.XPATH, ".//div[contains(@class, 'r8Zk0bSoBfwe23GKUAiO')]")
                    sub_date_str = sub_date_elements[0].text if sub_date_elements else "날짜 정보 없음"

                    temp_list.append({
                        "언론사": sub_press, "날짜_원문": sub_date_str, "제목": sub_title,
                        "URL": sub_href, "구분": "관련 뉴스"
                    })
                    processed_urls.add(sub_href)

        except NoSuchElementException:
            # 관련 뉴스가 없는 경우 통과
            pass

    except Exception:
        continue

    # 중간 저장 로직
    # save_interval의 배수가 되거나, 마지막 기사 덩어리일 때 저장
    if (i + 1) % save_interval == 0 or (i + 1) == len(final_article_clusters):
        if temp_list: # 임시 리스트에 데이터가 있을 경우에만 저장
            print(f"-> 기사 덩어리 {i + 1}개 처리 완료. {len(temp_list)}개의 새 정보를 파일에 중간 저장합니다...")
            temp_df = pd.DataFrame(temp_list)

            if is_first_save:
                temp_df.to_csv(backup_filename, encoding="utf-8-sig", index=False, mode='w')
                is_first_save = False
            else:
                temp_df.to_csv(backup_filename, encoding="utf-8-sig", index=False, mode='a', header=False)

            temp_list = []

# --- 최종 결과 처리 ---
browser.quit()

try:
    final_df = pd.read_csv(backup_filename)
    print("\n" + "="*50)
    print(f"총 {len(final_df)}개의 기사 정보(Sub 기사 포함)를 성공적으로 수집했습니다.")
    print("="*50)
    display(final_df)
except FileNotFoundError:
    print("\n수집된 기사 정보가 없습니다.")

## [Phase 1-2] 네이버 블로그 URL 수집

1. 프로젝트 개요

해당 파일은 '카카오톡 업데이트 사용자 피드백 분석' 프로젝트의 데이터 수집 단계 중, 네이버 블로그의 게시물 URL을 수집하는 과정을 담고 있습니다.

네이버 검색 결과 페이지의 동적 로딩 특성에 대응하기 위해 Selenium을 활용하여 자동화 스크립트를 구축했습니다.

2. 주요 로직 및 문제 해결 과정

네이버 블로그 검색 결과 페이지는 사용자가 스크롤을 내릴 때마다 새로운 게시물이 동적으로 로드되는 구조를 가지고 있습니다.

안정적인 대량 URL 수집을 위해 다음과 같은 전략을 적용했습니다.

- 자동 스크롤링: Selenium의 execute_script를 사용하여, 목표한 수집량에 도달하거나 페이지의 끝에 닿을 때까지 자동으로 스크롤을 반복 실행하여 모든 게시물을 로드합니다.
- 중복 방지: 수집 과정에서 동일한 URL이 중복으로 저장되는 것을 방지하기 위해, set() 자료구조를 활용하여 이미 처리된 URL은 건너뛰도록 설계했습니다.
- 정확한 타겟팅: 네이버의 주기적인 HTML 구조 변경에 대응하기 위해, XPath의 contains 함수를 사용하여 특정 패턴을 가진 클래스 이름을 유연하게 찾아내는 방식으로 목표 요소를 특정했습니다.
- 오류 제어: 스크롤 중 예기치 않은 오류가 발생하더라도 전체 작업이 중단되지 않도록 try-except 구문을 적용하고, 디버깅을 위해 오류 발생 시점의 페이지 소스를 저장하는 로직을 포함했습니다.

3. 결과물

스크립트 내 입력한 키워드별로 '작성자, 날짜, 제목, URL' 정보가 포함된 CSV 파일이 생성됩니다.

In [ ]:
import time
import pandas as pd

# 드라이버 경로를 지정 및 실행
browser = webdriver.Chrome(options=options)
wait = WebDriverWait(browser, 10)

# 검색 키워드 입력
keyword = "카카오톡 업데이트"
max_posts_to_collect = 2000

collected_posts = []
processed_urls = set()

print(f"'{keyword}' 키워드로 네이버 '블로그' 탭에서| 최대 {max_posts_to_collect}개의 게시물 정보 수집을 시작합니다.")

url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={keyword}"

browser.get(url)
time.sleep(2)

# --- 스크롤 Loop ---
while len(collected_posts) < max_posts_to_collect:
    try:
        last_height = browser.execute_script("return document.body.scrollHeight")

        # 게시물 블록 코드 설정
        wait.until(EC.presence_of_element_located(
            # mfeHMC8S51Ze981dhxwE > Orf2UUyw2B80LaIjeSZl > Cp6RskavnKnCD7vKgZWE > NtKCZYlcjvHdeUoASy2I > xYjt3uiECoJ0o6Pj0xOU > Hfu47jvQS6pqdbKB6Rpc > oIxNPKojSTvxvkjdwXVC
            # ehKpiBNGSFhS0YAl77ql > UFCFF04iBeRqybsMCKOx > PwscyUqz7vsmI7BTtlKF > Q752ck25tDSrEHVrEwXu > h5EzM_og0Ma9AHxIEpcW > YOhidyWXAbmCpDK3cyOc
            (By.XPATH, "//div[contains(@class, 'VU0scrRNz7gCayloyXne')]")
        ))
        # 게시물 블록 코드를 그대로 입력
        post_blocks = browser.find_elements(By.XPATH, "//div[contains(@class, 'VU0scrRNz7gCayloyXne')]")

        print(f"스크롤 다운... 현재 로드된 게시물 수: {len(post_blocks)}개. (수집 목표: {max_posts_to_collect}개)")

        start_index = len(processed_urls)

        newly_loaded_blocks = post_blocks[start_index:]

        for item in newly_loaded_blocks:
            if len(collected_posts) >= max_posts_to_collect:
                break
            try:
                # 제목/링크 <a> 태그 설정
                # Pcw4FFPrGxhURyUmBGxh > M9lOyC5Ckpmk7fKVCMeD > pHHExKwXvRWn4fm5O0Hr > z1n21OFoYx6_tGcWKL_x > CC5p8OBUeZzCymeWTg7v > FRu1e9uGplhm8_kzolkC > VL4Ep4IHjrVh0IEZTTPu
                # XAGjLtLX_Jwt7Ucd8crh > Cl492OuZm1euNoyyjv4M > kBKqMEieFEgrAIYJK3vg > vnDTNtnoko3GR0aJilUy > zsOIyFgaikMtT9gmM_tR > wi0982ME6rRJ9toqt_l_
                title_element = item.find_element(By.XPATH, ".//a[contains(@class, 'WguH38209auzUF3e7wuS')]")
                href = title_element.get_attribute('href')

                if href not in processed_urls:
                    title = title_element.text

                    # 작성자는 'sds-comps-profile-info-title-text' 클래스를 포함하는 <span> 태그 - 변경 없음
                    author = item.find_element(By.XPATH, ".//span[contains(@class, 'sds-comps-profile-info-title-text')]").text

                    # 날짜는 'sds-comps-profile-info-subtext' 클래스를 가진 <span> (기존과 동일) - 변경 없음
                    date_str = item.find_element(By.XPATH, ".//span[contains(@class, 'sds-comps-profile-info-subtext')]").text


                    collected_posts.append({
                        "작성자": author,
                        "날짜": date_str,
                        "제목": title_element.text,
                        "URL": href
                    })
                    processed_urls.add(href)
            except Exception:
                continue

        if len(collected_posts) >= max_posts_to_collect:
            print(f"목표 수집량({max_posts_to_collect}개)에 도달하여 스크롤을 중단합니다.")
            break

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            print("페이지의 끝에 도달하여 스크롤을 중단합니다.")
            break

    except Exception as e:
        print(f"스크롤 중 오류 발생: {e}")
        # 오류 발생 시 디버깅 파일 저장
        print("디버깅을 위해 현재 페이지 소스를 저장합니다.")
        with open('debug_error_page.html', 'w', encoding='utf-8') as f:
            f.write(browser.page_source)
        break

# --- 최종 결과 확인 ---
if collected_posts:
    df_links = pd.DataFrame(collected_posts)
    print("\n" + "="*50)
    print(f"총 {len(df_links)}개의 게시물 정보를 수집했습니다.")
    print("="*50)
    display(df_links)
else:
    print("\n수집된 게시물 정보가 없습니다.")

browser.quit()